In [1]:
import TPLC
import pandas as pd
import pyodbc 

GetAccessToken() called.
GetAccessToken() took 0.19 seconds.	Status Code: 200


In [2]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=B4BSQL;'
                      'Database=DynamicsAX_Prod;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
sql_query = '''
SELECT
	wms.WMSLOCATIONID					[Location]
	,ISNULL(SUM(ins.PHYSICALINVENT),0)	[Qty]
FROM WMSLOCATION wms
LEFT JOIN INVENTDIM ind ON ind.WMSLOCATIONID = wms.WMSLOCATIONID AND ind.DATAAREAID = wms.DATAAREAID
LEFT JOIN INVENTSUM ins ON ins.INVENTDIMID = ind.INVENTDIMID and ins.DATAAREAID = ind.DATAAREAID
WHERE wms.DATAAREAID = 'ssl'
  AND wms.INVENTLOCATIONID in ('Progress')
  AND LEFT(wms.WMSLOCATIONID, 1) = 'N'
GROUP BY wms.INVENTLOCATIONID, wms.WMSLOCATIONID
ORDER BY wms.INVENTLOCATIONID, wms.WMSLOCATIONID
'''

dax_data = pd.read_sql_query(sql_query, conn, index_col='Location')

In [3]:
# Collect 3PL Data
locations_raw = TPLC.GetLocations(pgsiz=10000, rql="facilityIdentifier.id==2;deactivated==False")
locations = {location["name"]: location["hasInventory"] for location in locations_raw}

# Convert 3PL Data into DataFrame
tpl_data = pd.DataFrame.from_dict(locations, orient='index')
tpl_data.index.name = 'Location'

Gathering page 1.	0 out of 6873 [0.00%]


In [17]:
data = tpl_data.merge(dax_data, how='left', left_index=True, right_index=True)

In [18]:
data

,0,Qty
Location,,
N05W12,False,96.0
N05W11,False,12.0
N05W10,False,26.0
N05W09,False,13.0
N05W08,False,43.0
...,...,...
SW02A05,False,NaN
SW02A04,True,NaN
SW02A03,True,NaN
